# Run

Here are some different options associated with running Docker containers.

## Memory

The `-m` option allows you to specify the memory limit for the container. For more detailed information, refer to the [official documentation page](https://docs.docker.com/engine/containers/resource_constraints/#memory).

---

The following example launches two containers: one with the default `-m` option and another with `-m 1g`. It then displays the output of the `docker stats` command. 

In [1]:
docker run -itd --rm --name memory_full alpine &> /dev/null
docker run -itd --rm -m 1g --name memory_limit alpine &> /dev/null

docker stats --no-stream

docker stop memory_full memory_limit &> /dev/null

CONTAINER ID   NAME           CPU %     MEM USAGE / LIMIT   MEM %     NET I/O       BLOCK I/O   PIDS
c697ea033dd4   memory_limit   0.00%     500KiB / 1GiB       0.05%     266B / 0B     0B / 0B     1
b0c9337d2eb0   memory_full    0.00%     500KiB / 31.01GiB   0.00%     2.25kB / 0B   0B / 0B     1


Notice that in the `MEM USAGE / LIMIT` column for the `memory_limit` container, the value is `1GiB`.

## CPUs

The `--cpus` option of the `docker run` command allows you to specify the number of CPU cores that will be allocated to the running Docker container.

---

As an example, consider an Alpine Docker container running the `stress-ng --cpu 3` command, which forces the system to utilize 3 CPU cores.

The following cell runs this container without any restrictions on the number of CPUs it can utilize.

In [7]:
docker run -id --rm --name cpu_experiments alpine sh -c \
    "apk update && apk add stress-ng && stress-ng --cpu 3" &> /dev/nbull
sleep 3
docker stats --no-stream
docker stop cpu_experiments &> /dev/null

CONTAINER ID   NAME              CPU %     MEM USAGE / LIMIT     MEM %     NET I/O           BLOCK I/O   PIDS
a310ffc6534e   cpu_experiments   301.56%   49.14MiB / 31.01GiB   0.15%     5.77MB / 75.6kB   0B / 0B     4


In the `docker stats` output, we observe 300% under the `CPU` column, indicating that all three CPU cores are being fully utilized. 

Now, we'll run a similar example, but with the key difference of using the `--cpus 2` option in the `docker run` command. 

In [9]:
docker run -id --rm --name cpu_experiments --cpus 2 alpine sh -c \
    "apk update && apk add stress-ng && stress-ng --cpu 3" &> /dev/null
sleep 3
docker stats --no-stream
docker stop cpu_experiments &> /dev/null &> /dev/null

CONTAINER ID   NAME              CPU %     MEM USAGE / LIMIT     MEM %     NET I/O          BLOCK I/O   PIDS
78c586383384   cpu_experiments   200.44%   49.13MiB / 31.01GiB   0.15%     5.86MB / 138kB   0B / 0B     4


Although `stress-ng` tries to use 3 CPU cores, in reality docker does not allow more than 2 cores.